In [3]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"location":"Paris, France"}', call_id='call_zseZCNwp4IA1s9Dc8EuA8Lyc', name='get_weather', type='function_call', id='fc_68040508d8288192b930c012d459fd6f0cc15c2f5b761137', status='completed')]


In [4]:
response.output

[ResponseFunctionToolCall(arguments='{"location":"Paris, France"}', call_id='call_zseZCNwp4IA1s9Dc8EuA8Lyc', name='get_weather', type='function_call', id='fc_68040508d8288192b930c012d459fd6f0cc15c2f5b761137', status='completed')]